# Membership Inference over Diffusion-models-based Synthetic Tabular Data (MIDST) Challenge @ SaTML 2025.

## Black Box Single Table Competition
Welcome to the MIDST challenge!

The MIDST challenge is a multi-track competition aiming to quantitatively evaluate the privacy of synthetic tabular data generated by diffusion models, with a specific focus on its resistance to membership inference attacks (MIAs).

This competition focuses on Black Box MIA on tabular diffusion models trained on a single table transaction dataset. The schema of the transaction dataset is as follows:
| trans_id | account_id | trans_date | trans_type | operation | amount  | balance  | k_symbol | bank | account |
|----------|------------|------------|------------|-----------|---------|----------|----------|------|---------|
| integer  | integer    | integer    | integer    | integer   | float   | float    | integer  | integer | integer |


MIA will be explored over two state-of-the-art methods [TabSyn](https://arxiv.org/pdf/2310.09656) and [TabDDPM](https://arxiv.org/pdf/2209.15421). A collection of TabSyn and TabDDPM models will be trained on random subsets of the transaction dataset. The goal is to create an approach (MIA) that can distinguish between samples used to train a model (train data) and other data randomly sampled from the transaction dataset (holdout data) given only output synthetic data from the model. The `final` set includes 20 models, each with its own set of challenge points (ie train and holdout data), to evaluate solutions on. To facilitate designing an attack, 30 `train` models are provided with comprehensive information about the model, training data and output synthetic data. Additionally, 20 `dev` models are provided to assist in evaluating the effectiveness of attacks prior to making a final submission to the `final` set. Participants can choose to perform MIA over one of or both TabSyn and TabDDPM. In the case of both, the attack that obtains the highest score will be used to rank the submission. A high level summary of the competition is below:
![bbox_diagram_final](https://github.com/user-attachments/assets/aa819365-2f2c-4dd9-a056-4ddb9b44aef7)

This notebook will walk you through the process of creating and packaging a submission to the black box single table challenge.

## Package Imports and Evironment Setup

Ensure that you have installed the proper dependenices to run the notebook. The environment installation instructions are available [here](https://github.com/VectorInstitute/MIDSTModels/tree/main/starter_kits). Now that we have verfied we have the proper packages installed, lets import them and define global variables:

In [1]:
import csv
import os
import random
import zipfile

from pathlib import Path
from functools import partial
from typing import Callable, Any

import numpy as np
import torch

from tqdm.notebook import tqdm
from data import get_challenge_points
from metrics import get_tpr_at_fpr

In [2]:
TABDDPM_DATA_DIR = "tabddpm_black_box"
TABSYN_DATA_DIR = "tabsyn_black_box"

## Data

Next, lets download and extract the data for the competition:

In [5]:
# Download and unzip tabddpm data
#!gdown 15-CY91tLyQ7mz1ghA9-1M_0khvu3DIX9
!unzip -qq -o tabddpm_black_box.zip

**Note:** If there is an issue with the download (ie throttled for downloading too many files with gdown) you can simply download the zip manually from this [link](https://drive.google.com/file/d/15-CY91tLyQ7mz1ghA9-1M_0khvu3DIX9/view?usp=drive_link) and extract it in the same directory this notebook exists.

In [6]:
# Download and unzip tabsyn data
#!gdown 1v5-iLpqun5uh4E0U1NREUwzgrV25_n3u
!unzip -qq -o tabsyn_black_box.zip

**Note:** If there is an issue with the download (ie throttled for downloading too many files with gdown) you can simply download the zip manually from this [link](https://drive.google.com/file/d/1v5-iLpqun5uh4E0U1NREUwzgrV25_n3u/view?usp=drive_link) and extract it in the same directory this notebook exists.

### Contents
The archives extracted under the `tabddpm_black_box` and `tabsyn_black_box` each contain 3 subdirectories:

- `train`: Comprehensive information (ie model weights+architecture, training data, output synthetic data etc.) about the set of shadow models. Use these to develop your attacks without having to train your own models.
- `dev`: Set of challenge points. Membership predictions for these challenges will be used to evaluate submissions during the competition and update the live scoreboard in CodaBench.
- `final`: Set of challenge points. Membership predictions for these challenges will be used to evaluate submissions when the competition closes and to determine the final ranking.

The contents of the `train` subdirectory of `tabddpm_black_box` and `tabsyn_black_box` slightly differ - this stems from the fact that each approach has its own set of artifacts from training. Below we outline the contents of the `train` for both TabSyn and TabDDPM along with the `dev` and `final` which include the same file types for each method.

<table> <tr> <th>Model - Stage</th> <th>File Name</th> <th>Description</th> </tr> <!-- TabDDPM - Train --> <tr> <td rowspan="8"><strong>TabDDPM - Train</strong></td> <td>train_with_id.csv</td> <td>Transaction samples used to train the model</td> </tr> <tr> <td>trans_domain.json</td> <td>Transaction data domain file indicating the domain information for each column</td> </tr> <tr> <td>challenge_with_id.csv</td> <td>Challenge points sampled from train data and holdout data</td> </tr> <tr> <td>challenge_label.csv</td> <td>The labels for the set of challenge points</td> </tr> <tr> <td>trans_label_encoders.pkl</td> <td>Pickled label encoders used in data preprocessing</td> </tr> <tr> <td>cluster_ckpt.pkl</td> <td>Pickled cluster checkpoint (Clustering does not impact training or sampling in single-table scenarios; the checkpoint exists solely because the multi-table pipeline was used for TabDDPM model.)</td> </tr> <tr> <td>None_trans_ckpt.pkl</td> <td>Pickled checkpoint of trained TabDDMP model</td> </tr> <tr> <td>trans_synthetic.csv</td> <td>Synthetic transaction data generated using the trained model</td> </tr> <!-- TabSyn - Train --> <tr> <td rowspan="6"><strong>TabSyn - Train</strong></td> <td>train_with_id.csv</td> <td>Transaction samples used to train the model</td> </tr> <tr> <td>challenge_with_id.csv</td> <td>Challenge points sampled from train data and holdout data</td> </tr> <tr> <td>challenge_label.csv</td> <td>The labels for the set of challenge points</td> </tr> <tr> <td>model.pt</td> <td>Pickled checkpoint of trained diffusion model of Tabsyn</td> </tr> <tr> <td>vae/</td> <td>Model artifacts for trained VAE model of TabSyn</td> </tr> <tr> <td>trans_synthetic.csv</td> <td>Synthetic transaction data generated using the trained model</td> </tr> <!-- TabDDPM/TabSyn - Dev --> <tr> <td rowspan="2"><strong>TabDDPM/TabSyn - Dev</strong></td> <td>challenge_with_id.csv</td> <td>Challenge points sampled from train data and holdout data</td> </tr> <tr> <td>trans_synthetic.csv</td> <td>Synthetic transaction data generated using the trained model</td> </tr> <!-- TabDDPM/TabSyn - Eval --> <tr> <td rowspan="2"><strong>TabDDPM/TabSyn - Final</strong></td> <td>challenge_with_id.csv</td> <td>Challenge points sampled from train data and holdout data</td> </tr> <tr> <td>trans_synthetic.csv</td> <td>Synthetic transaction data generated using the trained model</td> </tr> </table>




In [15]:
import pandas as pd
train_with_id = pd.read_csv('/data8/projets/dev_synthetic_data/code/lherbault/MIDSTModels/starter_kits/tabddpm_black_box/train/tabddpm_1/train_with_id.csv',
           )
trans_synthetic = pd.read_csv('/data8/projets/dev_synthetic_data/code/lherbault/MIDSTModels/starter_kits/tabddpm_black_box/train/tabddpm_1/trans_synthetic.csv')
trans_domain = pd.read_json('/data8/projets/dev_synthetic_data/code/lherbault/MIDSTModels/starter_kits/tabddpm_black_box/train/tabddpm_1/trans_domain.json')

In [16]:
trans_domain

,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account
size,2191,3,6,40400,542739,9,14,7665
type,continuous,discrete,discrete,continuous,continuous,discrete,discrete,continuous


In [12]:
train_with_id.head()

,trans_id,account_id,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account
0,707363,2423,1377,2,1,1494.0,14377.4,5,6,35182595
1,867043,2951,1154,2,4,14.6,35891.2,6,0,0
2,417654,1416,282,2,4,37800.0,66826.9,1,0,0
3,619464,2114,2098,2,4,5400.0,17284.3,1,0,0
4,188479,638,2048,2,1,2487.0,28732.4,5,9,41244357


In [14]:
trans_synthetic.head()

,trans_date,trans_type,operation,amount,balance,k_symbol,bank,account
0,1349.751636,2,1,2000.000000,20082.885949,5,3,1.435797e+07
1,907.060206,0,3,12953.732745,47206.669127,1,0,0.000000e+00
2,1915.000000,2,4,15929.424313,55285.090155,1,0,0.000000e+00
3,2129.000000,2,4,6521.817407,30213.488368,1,0,0.000000e+00
4,2003.892563,0,0,108.819056,25525.403622,7,0,0.000000e+00


1. entrainer tabddpm pour generer une deuxieme generation de donnees synthetiques (train = trans_synthetic) soit dans dev soit dans train:
   - le modele est dans le dossier single_table_tabddpm (notebook dispo)
   - le charger puis l'entrainer sur le trans_synthetic (train + garder un test pour evaluer la qualite)
   - evaluer la qualite
2. entrainer MIA pour differencier la premiere de la deuxieme generation de donnees synthetiques
3. evaluer sur le dataset dev

## Task

Your task as a competitor is to produce, for each model in `dev` and `final` in `tabddpm_black_box` and `tabsyn_black_box`, a CSV file listing your confidence scores (values between 0 and 1) for the membership of the challenge examples. You must save these scores in a `prediction.csv` file and place it in the same folder as the corresponding model. A submission to the challenge is an an archive containing just these `prediction.csv` files.

**You must submit predictions for both `dev` and `final` when you submit to CodaBench.**

In the following, we will show you how to correctly package a submission to the competition. To focus solely on the submission logic, the attack model will simply generate random predictions. Let's start by creating baseline attack models `tabddpm_attack_model` and `tabsyn_attack_model` based on their respective shadow models: 

In [7]:
def get_attack_model(base_train_path: Path) -> Callable[[Any], float]:
    return lambda x : random.uniform(0, 1)

base_tabddpm_train_path = os.path.join(TABDDPM_DATA_DIR, "train")
base_tabsyn_train_path = os.path.join(TABSYN_DATA_DIR, "train")
tabddpm_attack_model = get_attack_model(base_tabddpm_train_path)
tabsyn_attack_model = get_attack_model(base_tabsyn_train_path)

Using the attack model, we can obtain predictions for each point in the challenge point set for train, dev and final:

In [6]:
phases = ["train", "dev", "final"]

for base_dir, attack_model in zip([TABDDPM_DATA_DIR, TABSYN_DATA_DIR], [tabddpm_attack_model, tabsyn_attack_model]):
    for phase in phases:
        root = os.path.join(base_dir, phase)
        model_folders = [item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item))]
        for model_folder in sorted(model_folders, key=lambda d: int(d.split('_')[1])):
            path = os.path.join(root, model_folder)
    
            challenge_points = get_challenge_points(path)
    
            predictions = torch.Tensor([attack_model(cp) for cp in challenge_points])
           
            assert torch.all((0 <= predictions) & (predictions <= 1))
            with open(os.path.join(path, "prediction.csv"), mode="w", newline="") as file:
                writer = csv.writer(file)
    
                # Write each value in a separate row
                for value in list(predictions.numpy().squeeze()):
                    writer.writerow([value])

## Scoring

Let's see how the attack does on `train`, for which we have the ground truth.
When preparing a submission, you can use part of `train` to develop an attack and a held-out part to evaluate your attack.

In [7]:
tpr_at_fpr_list = []
for base_dir in [TABDDPM_DATA_DIR, TABSYN_DATA_DIR]:
    predictions = []
    solutions  = []
    root = os.path.join(base_dir, "train")
    model_folders = [item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item))]
    for model_folder in sorted(model_folders, key=lambda d: int(d.split('_')[1])):
        path = os.path.join(root, model_folder)
        predictions.append(np.loadtxt(os.path.join(path, "prediction.csv")))
        solutions.append(np.loadtxt(os.path.join(path, "challenge_label.csv"), skiprows=1))
    
    predictions = np.concatenate(predictions)
    solutions = np.concatenate(solutions)
    
    tpr_at_fpr = get_tpr_at_fpr(solutions, predictions)
    tpr_at_fpr_list.append(tpr_at_fpr)
    
    print(f"{base_dir.split('_')[0]} Train Attack TPR at FPR==10%: {tpr_at_fpr}")

final_tpr_at_fpr = max(tpr_at_fpr_list)
print(f"Final Train Attack TPR at FPR==10%: {final_tpr_at_fpr}")

tabddpm Train Attack TPR at FPR==10%: 0.102
tabsyn Train Attack TPR at FPR==10%: 0.08266666666666667
Final Train Attack TPR at FPR==10%: 0.102


## Packaging the submission

Now we can store the predictions into a zip file, which you can submit to CodaBench. Importantly, we create a single zip file for dev and final. The structure of the submission is as follows:
```
└── root_folder
    ├── tabsyn_black_box
    │   ├── dev
    │   │   └── tabsyn_#
    │   │       └── prediction.csv
    │   └── final
    │       └── tabsyn_#
    │           └── prediction.csv
    └── tabddpm_black_box
        ├── dev 
        │   └── tabddpm_#
        │       └── prediction.csv
        └── final 
            └── tabddpm_# 
                └── prediction.csv
```

**Note:** The `root_folder` can have any name but it is important all of the subdirectories follow the above structure and naming conventions. 

If a participant is looking to submit an attack for only one of TabSyn and TabDDPM, they can simply omit the other directory (ie `tabddpm_black_box` or `tabsyn_black_box` from the root_folder).

In [9]:
with zipfile.ZipFile(f"black_box_single_table_submission.zip", 'w') as zipf:
    for phase in ["dev", "final"]:
        for base_dir in [TABDDPM_DATA_DIR, TABSYN_DATA_DIR]:
            root = os.path.join(base_dir, phase)
            model_folders = [item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item))]
            for model_folder in sorted(model_folders, key=lambda d: int(d.split('_')[1])):
                path = os.path.join(root, model_folder)
                if not os.path.isdir(path): continue

                file = os.path.join(path, "prediction.csv")
                if os.path.exists(file):
                    # Use `arcname` to remove the base directory and phase directory from the zip path
                    arcname = os.path.relpath(file, os.path.dirname(base_dir))
                    zipf.write(file, arcname=arcname)
                else:
                    raise FileNotFoundError(f"`prediction.csv` not found in {path}.")

The generated black_box_single_table_submission.zip can be directly submitted to the dev phase in the CodaBench UI. Although this submission contains your predictions for both the dev and final set, you will only receive feedback on your predictions for the dev phase. The predictions for the final phase will be evaluated once the competiton ends using the most recent submission to the dev phase.